<a href="https://colab.research.google.com/github/skmisht/CNN/blob/main/CNN_Caffe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Melanoma detection using - Caffe

In [1]:
#Connect google colab to google drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
#install caffe with GPU support
#!apt install -y caffe-cuda
#install caffe with CPU support
!apt install -y caffe-cpu

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libcublas9.1 libcudart9.1 libcurand9.1
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  caffe-tools-cpu libcaffe-cpu1 python3-caffe-cpu
Suggested packages:
  libcaffe-cpu-dev caffe-doc
The following packages will be REMOVED:
  caffe-cuda caffe-tools-cuda libcaffe-cuda1 python3-caffe-cuda
The following NEW packages will be installed:
  caffe-cpu caffe-tools-cpu libcaffe-cpu1 python3-caffe-cpu
0 upgraded, 4 newly installed, 4 to remove and 10 not upgraded.
Need to get 1,596 kB of archives.
After this operation, 5,944 kB disk space will be freed.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libcaffe-cpu1 amd64 1.0.0-6 [798 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 caffe-tools-cpu amd64 1.0.0-6 [109 kB]
Get:3 http://archive.ubuntu.com

In [35]:
import pandas as pd 

training_truth_data = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Part3_GroundTruth.csv') 
testing_truth_data = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Part3_GroundTruth.csv') 

training_metadata = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data/ISIC-2017_Training_Data_metadata.csv')
testing_metadata = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data_metadata.csv')

path_training_images ='/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data/'
path_testing_images ='/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data/'


#add new column to training_truth_data witht he path to corresponding image
path_to_training_images_array = []
for x in training_truth_data.values:
    path_to_image = path_training_images + x[0] +'.jpg'
    path_to_training_images_array.append(path_to_image)
#add path to image to the training data frame
training_truth_data['path_to_image'] = path_to_training_images_array
#merge training truth data with metadata, join on the image_id's
training_data = pd.merge(training_truth_data, training_metadata, left_on='image_id', right_on='image_id', how='left').drop('image_id', axis=1)

#add new column to test_truth_data witht he path to corresponding image
path_to_test_images_array = []
for x in testing_truth_data.values:
  path_to_image = path_testing_images + str(x[0])+'.jpg'
  path_to_test_images_array.append(path_to_image)
#add path to image to the testing data frame
testing_truth_data['path_to_image'] = path_to_test_images_array
#merge testing truth data with metadata, join on the image_id's
testing_data = pd.merge(testing_truth_data, testing_metadata, left_on='image_id', right_on='image_id', how='left').drop('image_id', axis=1)


print(training_data.isnull().sum())
print(testing_data.isnull().sum())



melanoma                0
seborrheic_keratosis    0
path_to_image           0
age_approximate         0
sex                     0
dtype: int64
melanoma                0
seborrheic_keratosis    0
path_to_image           0
age_approximate         0
sex                     0
dtype: int64


In [27]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import caffe

#set mode to use GPU
caffe.set_mode_cpu()

#load network architecture
net = caffe.Net('/content/drive/MyDrive/Caffe/cnn_model.prototxt', caffe.TEST)

#2351622
input = None
for x in training_data.values:
  im = np.array(Image.open(x[2]))
  im_input = im[np.newaxis, np.newaxis, :, :]
  print(x[2] +'  '+str(im_input.size))
  if (input is None):
    input = im_input
  else:
    input = np.concatenate((input, im_input))

net.blobs['data'].reshape(*input.shape)
net.blobs['data'].data[...] = input

net.save('/content/drive/MyDrive/Caffe/melanoma_cnn.caffemodel')



/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data/ISIC_0000000.jpg  2351622
/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data/ISIC_0000001.jpg  2351622
/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data/ISIC_0000002.jpg  2351622
/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data/ISIC_0000003.jpg  2351622
/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data/ISIC_0000004.jpg  2351622
/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data/ISIC_0000006.jpg  2351622
/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data/ISIC_0000007.jpg  2351622
/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data/ISIC_0000008.jpg  2351622
/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Da

ValueError: ignored